In [1]:
import os
import pandas as pd
import numpy as np

# Check if you're running on Kaggle
IS_KAGGLE = os.path.exists("/kaggle/input")

if IS_KAGGLE:
    dataset_path = "/kaggle/input/cafe-sales-dirty-data-for-cleaning-training/dirty_cafe_sales.csv"
else:
    dataset_path = "datasets/dirty_cafe_sales.csv"

# Load the dataset
df = pd.read_csv(dataset_path)

# Replace 'unknown' and 'error' with NaN
display(df.head())

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [2]:
df.describe()
print(df[["Transaction ID", "Item"]].head(20))
print(df["Transaction ID"].value_counts().head(10))

   Transaction ID      Item
0     TXN_1961373    Coffee
1     TXN_4977031      Cake
2     TXN_4271903    Cookie
3     TXN_7034554     Salad
4     TXN_3160411    Coffee
5     TXN_2602893  Smoothie
6     TXN_4433211   UNKNOWN
7     TXN_6699534  Sandwich
8     TXN_4717867       NaN
9     TXN_2064365  Sandwich
10    TXN_2548360     Salad
11    TXN_3051279  Sandwich
12    TXN_7619095  Sandwich
13    TXN_9437049    Cookie
14    TXN_8915701     ERROR
15    TXN_2847255     Salad
16    TXN_3765707  Sandwich
17    TXN_6769710     Juice
18    TXN_8876618      Cake
19    TXN_3709394     Juice
Transaction ID
TXN_1961373    1
TXN_4831525    1
TXN_1228927    1
TXN_6486912    1
TXN_3447069    1
TXN_8219298    1
TXN_1010950    1
TXN_6376329    1
TXN_1897783    1
TXN_2767034    1
Name: count, dtype: int64


In [3]:
# Replace all case variations of 'unknown' and 'error' with NaN (null) without regex
to_null = lambda x: np.nan if str(x).lower() in ['unknown', 'error'] else x
df = df.applymap(to_null)
display(df.head())

/var/folders/6p/l4q7z30n5sv_r0vgy0ll38fh0000gn/T/ipykernel_24462/3868124749.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(to_null)


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,NaN,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,NaN,NaN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [4]:
display(df.dtypes)

Transaction ID      object
Item                object
Quantity            object
Price Per Unit      object
Total Spent         object
Payment Method      object
Location            object
Transaction Date    object
dtype: object

In [5]:
# Show all columns and their null value counts
display(df.isnull().sum())
display(df.columns)

Transaction ID         0
Item                 969
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

Index(['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date'],
      dtype='object')

In [6]:
modes = {col: df[col].mode().iloc[0] for col in df.columns if not df[col].mode().empty}
df.fillna(value=modes, inplace=True)

In [7]:
print("\nMissing values per column:")
display(df.isnull().sum())


Missing values per column:


Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

In [8]:
df.dtypes

Transaction ID      object
Item                object
Quantity            object
Price Per Unit      object
Total Spent         object
Payment Method      object
Location            object
Transaction Date    object
dtype: object

In [9]:
# Numeric columns you expect
num_cols = ["Quantity", "Price Per Unit", "Total Spent"]
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")  # string to float/int, invalids become NaN


# Date columns
date_cols = ["Transaction Date"]  # replace with real
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

# Categorical cleanup
cat_cols = ["Transaction ID", "Item", "Payment Method", "Location"]
for col in cat_cols:
    df[col] = df[col].astype("category")

In [10]:
df.dtypes

Transaction ID            category
Item                      category
Quantity                     int64
Price Per Unit             float64
Total Spent                float64
Payment Method            category
Location                  category
Transaction Date    datetime64[ns]
dtype: object

In [11]:
print("\nCheck for duplicates after cleaning:")
display(df.duplicated().sum())


Check for duplicates after cleaning:


np.int64(0)

In [12]:
from mlxtend.frequent_patterns import apriori, association_rules

basket = df.groupby(["Transaction ID", "Item"])["Item"].count().unstack().fillna(0)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

# 3. Apply Apriori
frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)

# 4. Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
print(rules[["antecedents", "consequents", "support", "confidence", "lift"]])

Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []


/var/folders/6p/l4q7z30n5sv_r0vgy0ll38fh0000gn/T/ipykernel_24462/4131895463.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  basket = df.groupby(["Transaction ID", "Item"])["Item"].count().unstack().fillna(0)
/var/folders/6p/l4q7z30n5sv_r0vgy0ll38fh0000gn/T/ipykernel_24462/4131895463.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)
/Users/jeritbaiju/workspaces/MCKA/feature-engineering-mcka/.venv/lib/python3.13/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
